レース環境情報をスクレイピングで取得

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm

In [ ]:
race_id = "202104020101"
url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
html = requests.get(url)
html.encoding= "EUC_JP"
soup = BeautifulSoup(html.text, "html.parser")

In [ ]:
日付 = soup.find("dd",class_='Active')
日付.find("a").get_text()

In [ ]:
レース情報 = soup.find("div", class_="RaceList_Item02") 

In [ ]:
レース名 = レース情報.find("div",class_="RaceName").get_text()
re.findall(r"\w+",レース名)

In [ ]:
レース環境 = レース情報.find("div",class_="RaceData01").get_text()
re.findall(r"\w+",レース環境)

In [ ]:
df = pd.read_pickle('df.pickle')
race_id_list = df.index.unique()
race_id_list

df.pickleに保存したdataframeのindexを参照してして全てのレース環境情報をスクレイピング

In [65]:
race_infos = {}
for race_id in tqdm(race_id_list):
  url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
  html = requests.get(url)
  html.encoding= "EUC_JP"
  soup = BeautifulSoup(html.text, "html.parser")

  日付 = soup.find("dd",class_='Active')
  日付.find("a").get_text()
  レース情報 = soup.find("div", class_="RaceList_Item02") 
  # 距離 = レース情報.find("div",class_="RaceData01").find("span").get_text()
  # 馬場 = レース情報.find("div",class_="RaceData01").find("span",class_="Item03").get_text()
  レース環境 = レース情報.find("div",class_="RaceData01").get_text()
  info = re.findall(r"\w+",レース環境)
  info[0] = info[2][:1]
  info[1] = info[2][1:]
  info.pop(2)
  info_dict = {}
  # print(info)
  for text in info:
    # print(text)
    if text in ["芝","ダ"]:
      # print("芝付き")
      info_dict["race_type"] = text
    if "障" in text:
      info_dict["race_type"] = "障害"
    if "m" in text:
      # print("mつき")
      info_dict["course_ren"] = re.findall(r"\d+",text)[0]
    if text in ["雨","小雨","小雪","雪","晴","曇"]:
      # print("天気")
      info_dict["weather"] = text
    # if "年" in text:
    #   info_dict["data"] = text
    if text in ["良","不良","重","稍","不"]:
      # print("馬場")
      info_dict["ground_state"] = text

  race_infos[race_id] = info_dict
  time.sleep(.5)

100%|██████████| 173/173 [02:37<00:00,  1.10it/s]


取得データの出力

In [67]:
# pd.set_option('display.max_rows',None)

pd.DataFrame(race_infos)

,202101010101,202101010102,202102010101,202102010102,202103010101,202103010102,202104010101,202104010102,202105010101,202105010102,...,202109040102,202109040201,202109040202,202109040301,202110040101,202110040102,202110040201,202110040202,202110040301,202110040302
race_type,芝,ダ,芝,ダ,芝,ダ,ダ,ダ,ダ,ダ,...,ダ,ダ,ダ,ダ,芝,芝,芝,ダ,芝,ダ
course_ren,1200,1700,1200,2400,2750,1700,1200,1800,1400,1400,...,1200,1800,1400,1800,2860,1200,1200,1700,1200,1700
weather,小雨,曇,晴,晴,曇,曇,晴,晴,晴,晴,...,晴,晴,晴,晴,雨,雨,晴,晴,曇,曇
ground_state,良,良,良,良,良,稍,良,良,重,重,...,良,良,良,良,不,不,重,不,良,良


In [68]:
pd.DataFrame(race_infos).T
# type(race_infos)

,race_type,course_ren,weather,ground_state
202101010101,芝,1200,小雨,良
202101010102,ダ,1700,曇,良
202102010101,芝,1200,晴,良
202102010102,ダ,2400,晴,良
202103010101,芝,2750,曇,良
202103010102,ダ,1700,曇,稍
202104010101,ダ,1200,晴,良
202104010102,ダ,1800,晴,良
202105010101,ダ,1400,晴,重
202105010102,ダ,1400,晴,重


In [ ]:
results = pd.read_pickle('df.pickle')
results

In [ ]:
results_addinfo = results.marge(race_infos,left_index=True,right_index=True)